<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/prediction_seed7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install pykalman

import numpy as np
# from tensorflow import keras
import pandas as pd
from scipy.signal import welch, butter, filtfilt
from pykalman import KalmanFilter
# import keras
from tensorflow.keras.models import load_model
import joblib

import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))


# EEG Channel List
channels = [
    "FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8",
    "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8",
    "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8",
    "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8",
    "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8",
    "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8",
    "CB1", "O1", "OZ", "O2", "CB2"
]

# Sampling rate and window size
sampling_rate_original = 2500  # Original EEG sampling rate
sampling_rate_final = 200  # After downsampling
window_size = 1  # 1 second
segment_length = sampling_rate_final * window_size

# EEG Frequency Bands
freq_bands = {
    "Delta": (1, 4),
    "Theta": (4, 8),
    "Alpha": (8, 14),
    "Beta": (14, 31),
    "Gamma": (31, 50)
}

# Kalman Filter initialization
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.2 * np.eye(1),
    observation_covariance=0.5 * np.eye(1),
    initial_state_mean=[20],
    initial_state_covariance=5 * np.eye(1)
)


# Helper: Bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y


# Helper: Downsample
def downsample_signal(signal, original_fs, target_fs):
    factor = int(original_fs / target_fs)
    return signal[::factor]


# Step 1: Generate high-sampling-rate EEG signal
def generate_extreme_positive_eeg():
    t = np.linspace(0, 1, sampling_rate_original)
    eeg_data = []

    for _ in range(62):
        # Random amplitude and frequency for each wave
        alpha_wave = np.random.uniform(40, 80) * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
        beta_wave = np.random.uniform(15, 35) * np.sin(2 * np.pi * np.random.uniform(14, 30) * t)
        delta_wave = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(1, 3) * t)
        theta_wave = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(4, 7) * t)
        gamma_wave = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(31, 50) * t)
        noise = np.random.normal(0, np.random.uniform(2, 5), t.shape)

        eeg_channel_signal = delta_wave + theta_wave + alpha_wave + beta_wave + gamma_wave + noise
        eeg_channel_signal = np.clip(eeg_channel_signal, -70, 70)

        eeg_data.append(eeg_channel_signal)

    return np.array(eeg_data)


# Step 2: Extract DE_LDS Features
def extract_de_lds():
    eeg_data = generate_extreme_positive_eeg()  # shape: (62, 2500)

    de_features = []

    for channel_data in eeg_data:
        # ✅ Downsample the channel from 2500Hz -> 200Hz
        downsampled_data = downsample_signal(channel_data, sampling_rate_original, sampling_rate_final)

        band_de = []
        for band, (low, high) in freq_bands.items():
            # ✅ Bandpass filter in the current band
            filtered_data = bandpass_filter(downsampled_data, low, high, sampling_rate_final, order=4)

            # ✅ Variance of the filtered signal
            variance = np.var(filtered_data)

            # ✅ Differential Entropy calculation (Gaussian assumption)
            de_value = 0.5 * np.log(2 * np.pi * np.e * variance + 1e-8)  # small epsilon to avoid log(0)

            band_de.append(de_value)

        band_de = np.array(band_de).reshape(-1, 1)

        # ✅ Apply Kalman Filter smoothing
        smoothed_de, _ = kf.filter(band_de)

        de_features.append(smoothed_de.flatten())

    de_features = np.array(de_features)  # shape (62, 5)
    # print(de_features)

    # ✅ Normalize DE values to match SEED range (15 to 27)
    de_min, de_max = np.min(de_features), np.max(de_features)
    seed_min, seed_max = 15, 27
    de_lds_values = seed_min + (de_features - de_min) / (de_max - de_min) * (seed_max - seed_min)

    print(f"🟢 Extracted DE_LDS Features - Shape: {de_lds_values.shape}")
    # print(de_lds_values.flatten())
    return de_lds_values.flatten()

# Step 3: Predict Emotion
def predict_emotion():
    sample = extract_de_lds().reshape((1,1,310))
    # sample = np.expand_dims(sample, axis=0)
    print(f"Sample shape: {sample.shape}")

    # ✅ Load model
    han_model = load_model("/content/eeg_emotion_bilstm_model(100e).h5")
    han_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # ✅ Load y_encoder using joblib
    y_encoder = joblib.load("/content/eeg_label_encoder.pkl")

    # ✅ Predict
    prediction = han_model.predict(sample)
    predicted_label = np.argmax(prediction, axis=1)
    decoded_label = y_encoder.inverse_transform(predicted_label)

    print(f"🔵 Predicted Emotion: {decoded_label[0]}")
    return decoded_label[0]

# Run it
predicted_emotion = predict_emotion()

Num GPUs Available: 1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
🟢 Extracted DE_LDS Features - Shape: (62, 5)
Sample shape: (1, 1, 310)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 664ms/step
🔵 Predicted Emotion: Surprise
